In [19]:
import pandas as pd
import numpy as np

In [30]:
matches = pd.read_csv("matches.csv", index_col = 0)

In [21]:
matches.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,2022-08-05,20:00,Premier League,Matchweek 1,Fri,Away,W,2.0,0.0,Crystal Palace,...,Match Report,NaN,10.0,2.0,14.6,1.0,0.0,0.0,2023,Arsenal
1,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4.0,2.0,Leicester City,...,Match Report,NaN,19.0,7.0,13.0,0.0,0.0,0.0,2023,Arsenal
2,2022-08-20,17:30,Premier League,Matchweek 3,Sat,Away,W,3.0,0.0,Bournemouth,...,Match Report,NaN,14.0,6.0,14.8,0.0,0.0,0.0,2023,Arsenal
3,2022-08-27,17:30,Premier League,Matchweek 4,Sat,Home,W,2.0,1.0,Fulham,...,Match Report,NaN,22.0,8.0,15.5,1.0,0.0,0.0,2023,Arsenal
4,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,2.0,1.0,Aston Villa,...,Match Report,NaN,22.0,8.0,16.3,1.0,0.0,0.0,2023,Arsenal


In [22]:
matches.shape

(1800, 27)

In [23]:
matches["team"].value_counts()

Leicester City              92
Newcastle United            92
Tottenham Hotspur           92
Wolverhampton Wanderers     92
Everton                     92
West Ham United             92
Aston Villa                 92
Crystal Palace              91
Leeds United                91
Manchester City             91
Arsenal                     91
Chelsea                     91
Brighton and Hove Albion    91
Liverpool                   91
Manchester United           91
Southampton                 76
Burnley                     76
Brentford                   54
Fulham                      54
Watford                     38
Norwich City                38
West Bromwich Albion        38
Sheffield United            38
Bournemouth                 16
Name: team, dtype: int64

In [24]:
matches["round"].value_counts()

Matchweek 1     58
Matchweek 10    58
Matchweek 2     58
Matchweek 17    58
Matchweek 15    58
Matchweek 14    58
Matchweek 13    58
Matchweek 11    58
Matchweek 16    58
Matchweek 9     58
Matchweek 6     58
Matchweek 5     58
Matchweek 4     58
Matchweek 3     58
Matchweek 12    56
Matchweek 8     52
Matchweek 34    40
Matchweek 31    40
Matchweek 32    40
Matchweek 30    40
Matchweek 33    40
Matchweek 35    40
Matchweek 36    40
Matchweek 37    40
Matchweek 28    40
Matchweek 29    40
Matchweek 20    40
Matchweek 27    40
Matchweek 26    40
Matchweek 25    40
Matchweek 24    40
Matchweek 23    40
Matchweek 22    40
Matchweek 21    40
Matchweek 19    40
Matchweek 18    40
Matchweek 7     40
Matchweek 38    40
Name: round, dtype: int64

In [25]:
matches.dtypes

date             object
time             object
comp             object
round            object
day              object
venue            object
result           object
gf              float64
ga              float64
opponent         object
xg              float64
xga             float64
poss            float64
attendance      float64
captain          object
formation        object
referee          object
match report     object
notes           float64
sh              float64
sot             float64
dist            float64
fk              float64
pk              float64
pkatt           float64
season            int64
team             object
dtype: object

In [31]:
# Convert date column into date type

matches["date"] = pd.to_datetime(matches["date"])

In [32]:
# Create hour column

matches["hour"] = matches["time"].str.replace(":.+", "", regex = True).astype("int")

In [33]:
# Convert text data to numbers

matches["venue_code"] = matches["venue"].astype("category").cat.codes
matches["opp_code"] = matches["opponent"].astype("category").cat.codes
matches["day_code"] = matches["date"].dt.dayofweek

In [34]:
matches.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,dist,fk,pk,pkatt,season,team,hour,venue_code,opp_code,day_code
0,2022-08-05,20:00,Premier League,Matchweek 1,Fri,Away,W,2.0,0.0,Crystal Palace,...,14.6,1.0,0.0,0.0,2023,Arsenal,20,0,7,4
1,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4.0,2.0,Leicester City,...,13.0,0.0,0.0,0.0,2023,Arsenal,15,1,11,5
2,2022-08-20,17:30,Premier League,Matchweek 3,Sat,Away,W,3.0,0.0,Bournemouth,...,14.8,0.0,0.0,0.0,2023,Arsenal,17,0,2,5
3,2022-08-27,17:30,Premier League,Matchweek 4,Sat,Home,W,2.0,1.0,Fulham,...,15.5,1.0,0.0,0.0,2023,Arsenal,17,1,9,5
4,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,2.0,1.0,Aston Villa,...,16.3,1.0,0.0,0.0,2023,Arsenal,19,1,1,2


In [35]:
# Random Forest

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)
train = matches[matches["date"] < '2022-09-01']
test = matches[matches["date"] > '2022-09-01' ]
predictors = ['venue_code','opp_code','hour','day_code']
rf.fit(train[predictors], train["result"])
preds = rf.predict(test[predictors])

In [37]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(test["result"], preds)
acc

0.45263157894736844

In [38]:
combined = pd.DataFrame(dict(actual=test["result"],prediction = preds))
pd.crosstab(index = combined['actual'],columns = combined['prediction'])

prediction,D,L,W
actual,,,
D,3,14,18
L,5,43,24
W,13,30,40


In [43]:
from sklearn.metrics import precision_score

precision_score(test['result'], preds, average = 'weighted')

0.42670532512947634

In [47]:
# Create dataframe for each squard

group_matches = matches.groupby('team')

In [50]:
group_matches.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,dist,fk,pk,pkatt,season,team,hour,venue_code,opp_code,day_code
0,2022-08-05,20:00,Premier League,Matchweek 1,Fri,Away,W,2.0,0.0,Crystal Palace,...,14.6,1.0,0.0,0.0,2023,Arsenal,20,0,7,4
1,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4.0,2.0,Leicester City,...,13.0,0.0,0.0,0.0,2023,Arsenal,15,1,11,5
2,2022-08-20,17:30,Premier League,Matchweek 3,Sat,Away,W,3.0,0.0,Bournemouth,...,14.8,0.0,0.0,0.0,2023,Arsenal,17,0,2,5
3,2022-08-27,17:30,Premier League,Matchweek 4,Sat,Home,W,2.0,1.0,Fulham,...,15.5,1.0,0.0,0.0,2023,Arsenal,17,1,9,5
4,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,2.0,1.0,Aston Villa,...,16.3,1.0,0.0,0.0,2023,Arsenal,19,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2020-09-14,18:00,Premier League,Matchweek 1,Mon,Home,L,0.0,2.0,Wolves,...,12.9,0.0,0.0,0.0,2022,Sheffield United,18,1,24,0
2,2020-09-21,18:00,Premier League,Matchweek 2,Mon,Away,L,0.0,1.0,Aston Villa,...,20.2,0.0,0.0,1.0,2022,Sheffield United,18,0,1,0
3,2020-09-27,12:00,Premier League,Matchweek 3,Sun,Home,L,0.0,1.0,Leeds United,...,16.7,0.0,0.0,0.0,2022,Sheffield United,12,1,10,6
4,2020-10-04,14:00,Premier League,Matchweek 4,Sun,Away,L,1.0,2.0,Arsenal,...,24.7,0.0,0.0,0.0,2022,Sheffield United,14,0,0,6


In [56]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("date")
    rolling_stats = group[cols].rolling(3, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset = new_cols)
    return group

In [52]:
cols = ['gf', 'ga', 'sh', 'sot', 'dist', 'fk', 'pk', 'pkatt']
new_cols = [f"{c}_rolling" for c in cols]

In [53]:
new_cols

['gf_rolling',
 'ga_rolling',
 'sh_rolling',
 'sot_rolling',
 'dist_rolling',
 'fk_rolling',
 'pk_rolling',
 'pkatt_rolling']

In [57]:
rolling_averages(group, cols, new_cols)

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,opp_code,day_code,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
6,2020-10-04,14:00,Premier League,Matchweek 4,Sun,Home,W,2.0,1.0,Sheffield Utd,...,18,6,2.000000,1.333333,8.000000,3.666667,14.633333,0.666667,0.000000,0.000000
7,2020-10-17,17:30,Premier League,Matchweek 5,Sat,Away,L,0.0,1.0,Manchester City,...,13,5,1.666667,1.666667,5.666667,3.666667,15.366667,0.000000,0.000000,0.000000
9,2020-10-25,19:15,Premier League,Matchweek 6,Sun,Home,L,0.0,1.0,Leicester City,...,11,6,1.000000,1.666667,7.000000,3.666667,16.566667,0.666667,0.000000,0.000000
11,2020-11-01,16:30,Premier League,Matchweek 7,Sun,Away,W,1.0,0.0,Manchester Utd,...,14,6,0.666667,1.000000,9.666667,4.000000,16.566667,1.000000,0.000000,0.000000
13,2020-11-08,19:15,Premier League,Matchweek 8,Sun,Home,L,0.0,3.0,Aston Villa,...,1,6,0.333333,0.666667,9.666667,2.666667,19.333333,1.000000,0.333333,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14,2022-10-23,14:00,Premier League,Matchweek 13,Sun,Away,D,1.0,1.0,Southampton,...,19,6,2.333333,1.000000,13.666667,6.333333,16.500000,0.666667,0.333333,0.333333
16,2022-10-30,14:00,Premier League,Matchweek 14,Sun,Home,W,5.0,0.0,Nott'ham Forest,...,17,6,1.666667,1.000000,10.333333,4.333333,16.033333,0.333333,0.333333,0.333333
18,2022-11-06,12:00,Premier League,Matchweek 15,Sun,Away,W,1.0,0.0,Chelsea,...,6,6,2.333333,0.333333,15.000000,5.333333,16.600000,0.666667,0.000000,0.000000
20,2022-11-12,19:45,Premier League,Matchweek 16,Sat,Away,W,2.0,0.0,Wolves,...,24,5,2.333333,0.333333,16.666667,4.666667,15.900000,0.333333,0.000000,0.000000


In [58]:
matches_rolling = matches.groupby("team").apply(lambda x:rolling_averages(x, cols, new_cols))
matches_rolling

date   time            comp         round  \
team                                                                         
Arsenal                 6  2020-10-04  14:00  Premier League   Matchweek 4   
                        7  2020-10-17  17:30  Premier League   Matchweek 5   
                        9  2020-10-25  19:15  Premier League   Matchweek 6   
                        11 2020-11-01  16:30  Premier League   Matchweek 7   
                        13 2020-11-08  19:15  Premier League   Matchweek 8   
...                               ...    ...             ...           ...   
Wolverhampton Wanderers 12 2022-10-23  14:00  Premier League  Matchweek 13   
                        13 2022-10-29  15:00  Premier League  Matchweek 14   
                        14 2022-11-05  15:00  Premier League  Matchweek 15   
                        16 2022-11-12  19:45  Premier League  Matchweek 16   
                        18 2022-12-26  15:00  Premier League  Matchweek 17   

                            day venue result   gf   ga         opponent  ...  \
team                                                                     ...   
Arsenal                 6   Sun  Home      W  2.0  1.0    Sheffield Utd  ...   
                        7   Sat  Away      L  0.0  1.0  Manchester City  ...   
                        9   Sun  Home      L  0.0  1.0   Leicester City  ...   
                        11  Sun  Away      W  1.0  0.0   Manchester Utd  ...   
                        13  Sun  Home      L  0.0  3.0      Aston Villa  ...   
...                         ...   ...    ...  ...  ...              ...  ...   
Wolverhampton Wanderers 12  Sun  Home      L  0.0  4.0   Leicester City  ...   
                        13  Sat  Away      D  1.0  1.0        Brentford  ...   
                        14  Sat  Home      L  2.0  3.0         Brighton  ...   
                        16  Sat  Home      L  0.0  2.0          Arsenal  ...   
                        18  Mon  Away      W  2.0  1.0          Everton  ...   

                            opp_code  day_code  gf_rolling  ga_rolling  \
team                                                                     
Arsenal                 6         18         6    2.000000    1.333333   
                        7         13         5    1.666667    1.666667   
                        9         11         6    1.000000    1.666667   
                        11        14         6    0.666667    1.000000   
                        13         1         6    0.333333    0.666667   
...                              ...       ...         ...         ...   
Wolverhampton Wanderers 12        11         6    0.666667    1.666667   
                        13         3         5    0.666667    2.000000   
                        14         4         5    0.666667    2.333333   
                        16         0         5    1.000000    2.666667   
                        18         8         0    1.000000    2.000000   

                           sh_rolling sot_rolling dist_rolling fk_rolling  \
team                                                                        
Arsenal                 6    8.000000    3.666667    14.633333   0.666667   
                        7    5.666667    3.666667    15.366667   0.000000   
                        9    7.000000    3.666667    16.566667   0.666667   
                        11   9.666667    4.000000    16.566667   1.000000   
                        13   9.666667    2.666667    19.333333   1.000000   
...                               ...         ...          ...        ...   
Wolverhampton Wanderers 12   8.666667    2.333333    15.866667   0.666667   
                        13  13.000000    3.333333    16.266667   0.666667   
                        14  14.666667    4.333333    17.466667   0.666667   
                        16  13.666667    4.333333    18.133333   0.333333   
                        18  10.333333    3.333333    19.566667   0.333333   

            

In [59]:
matches_rolling = matches_rolling.droplevel('team')
matches_rolling

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,opp_code,day_code,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
6,2020-10-04,14:00,Premier League,Matchweek 4,Sun,Home,W,2.0,1.0,Sheffield Utd,...,18,6,2.000000,1.333333,8.000000,3.666667,14.633333,0.666667,0.000000,0.000000
7,2020-10-17,17:30,Premier League,Matchweek 5,Sat,Away,L,0.0,1.0,Manchester City,...,13,5,1.666667,1.666667,5.666667,3.666667,15.366667,0.000000,0.000000,0.000000
9,2020-10-25,19:15,Premier League,Matchweek 6,Sun,Home,L,0.0,1.0,Leicester City,...,11,6,1.000000,1.666667,7.000000,3.666667,16.566667,0.666667,0.000000,0.000000
11,2020-11-01,16:30,Premier League,Matchweek 7,Sun,Away,W,1.0,0.0,Manchester Utd,...,14,6,0.666667,1.000000,9.666667,4.000000,16.566667,1.000000,0.000000,0.000000
13,2020-11-08,19:15,Premier League,Matchweek 8,Sun,Home,L,0.0,3.0,Aston Villa,...,1,6,0.333333,0.666667,9.666667,2.666667,19.333333,1.000000,0.333333,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12,2022-10-23,14:00,Premier League,Matchweek 13,Sun,Home,L,0.0,4.0,Leicester City,...,11,6,0.666667,1.666667,8.666667,2.333333,15.866667,0.666667,0.333333,0.333333
13,2022-10-29,15:00,Premier League,Matchweek 14,Sat,Away,D,1.0,1.0,Brentford,...,3,5,0.666667,2.000000,13.000000,3.333333,16.266667,0.666667,0.333333,0.333333
14,2022-11-05,15:00,Premier League,Matchweek 15,Sat,Home,L,2.0,3.0,Brighton,...,4,5,0.666667,2.333333,14.666667,4.333333,17.466667,0.666667,0.000000,0.000000
16,2022-11-12,19:45,Premier League,Matchweek 16,Sat,Home,L,0.0,2.0,Arsenal,...,0,5,1.000000,2.666667,13.666667,4.333333,18.133333,0.333333,0.333333,0.333333


In [60]:
matches_rolling.index = range(matches_rolling.shape[0])
matches_rolling

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,opp_code,day_code,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
0,2020-10-04,14:00,Premier League,Matchweek 4,Sun,Home,W,2.0,1.0,Sheffield Utd,...,18,6,2.000000,1.333333,8.000000,3.666667,14.633333,0.666667,0.000000,0.000000
1,2020-10-17,17:30,Premier League,Matchweek 5,Sat,Away,L,0.0,1.0,Manchester City,...,13,5,1.666667,1.666667,5.666667,3.666667,15.366667,0.000000,0.000000,0.000000
2,2020-10-25,19:15,Premier League,Matchweek 6,Sun,Home,L,0.0,1.0,Leicester City,...,11,6,1.000000,1.666667,7.000000,3.666667,16.566667,0.666667,0.000000,0.000000
3,2020-11-01,16:30,Premier League,Matchweek 7,Sun,Away,W,1.0,0.0,Manchester Utd,...,14,6,0.666667,1.000000,9.666667,4.000000,16.566667,1.000000,0.000000,0.000000
4,2020-11-08,19:15,Premier League,Matchweek 8,Sun,Home,L,0.0,3.0,Aston Villa,...,1,6,0.333333,0.666667,9.666667,2.666667,19.333333,1.000000,0.333333,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1720,2022-10-23,14:00,Premier League,Matchweek 13,Sun,Home,L,0.0,4.0,Leicester City,...,11,6,0.666667,1.666667,8.666667,2.333333,15.866667,0.666667,0.333333,0.333333
1721,2022-10-29,15:00,Premier League,Matchweek 14,Sat,Away,D,1.0,1.0,Brentford,...,3,5,0.666667,2.000000,13.000000,3.333333,16.266667,0.666667,0.333333,0.333333
1722,2022-11-05,15:00,Premier League,Matchweek 15,Sat,Home,L,2.0,3.0,Brighton,...,4,5,0.666667,2.333333,14.666667,4.333333,17.466667,0.666667,0.000000,0.000000
1723,2022-11-12,19:45,Premier League,Matchweek 16,Sat,Home,L,0.0,2.0,Arsenal,...,0,5,1.000000,2.666667,13.666667,4.333333,18.133333,0.333333,0.333333,0.333333


In [65]:
def make_predictions(data, predictors):
    train = data[data["date"] < '2022-09-01']
    test = data[data["date"] > '2022-09-01']
    rf.fit(train[predictors], train['result'])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test['result'],predicted=preds),index=test.index)
    precision = precision_score(test['result'],preds, average = 'weighted')
    return combined,precision

In [66]:
combined, precision = make_predictions(matches_rolling, predictors + new_cols)

In [67]:
precision

0.47235406698564597

In [68]:
combined

,actual,predicted
78,L,W
79,W,W
80,W,W
81,W,W
82,W,W
...,...,...
1720,L,L
1721,D,D
1722,L,W
1723,L,L


In [69]:
combined = combined.merge(matches_rolling[['date','team','opponent','result']],left_index = True, right_index = True)
combined

,actual,predicted,date,team,opponent,result
78,L,W,2022-09-04,Arsenal,Manchester Utd,L
79,W,W,2022-09-18,Arsenal,Brentford,W
80,W,W,2022-10-01,Arsenal,Tottenham,W
81,W,W,2022-10-09,Arsenal,Liverpool,W
82,W,W,2022-10-16,Arsenal,Leeds United,W
...,...,...,...,...,...,...
1720,L,L,2022-10-23,Wolverhampton Wanderers,Leicester City,L
1721,D,D,2022-10-29,Wolverhampton Wanderers,Brentford,D
1722,L,W,2022-11-05,Wolverhampton Wanderers,Brighton,L
1723,L,L,2022-11-12,Wolverhampton Wanderers,Arsenal,L


In [73]:
class MissingDict(dict):
    __missing__ = lambda self, key: key
    
map_values = {
    "Brighton and Have Albion": "Brighton",
    "Manchester United": "Manchester Utd",
    "Newcastle United": "Newcastle Utd",
    "Tottenham Hotspur": "Tottenham",
    "West Ham United": "West Ham",
    "Wolverhampton Wanderers": "Wolves"
}
mapping = MissingDict(**map_values)

In [75]:
combined['new_team'] = combined['team'].map(mapping)

In [76]:
combined

,actual,predicted,date,team,opponent,result,new_team
78,L,W,2022-09-04,Arsenal,Manchester Utd,L,Arsenal
79,W,W,2022-09-18,Arsenal,Brentford,W,Arsenal
80,W,W,2022-10-01,Arsenal,Tottenham,W,Arsenal
81,W,W,2022-10-09,Arsenal,Liverpool,W,Arsenal
82,W,W,2022-10-16,Arsenal,Leeds United,W,Arsenal
...,...,...,...,...,...,...,...
1720,L,L,2022-10-23,Wolverhampton Wanderers,Leicester City,L,Wolves
1721,D,D,2022-10-29,Wolverhampton Wanderers,Brentford,D,Wolves
1722,L,W,2022-11-05,Wolverhampton Wanderers,Brighton,L,Wolves
1723,L,L,2022-11-12,Wolverhampton Wanderers,Arsenal,L,Wolves


In [77]:
merged = combined.merge(combined, left_on=['date','new_team'],right_on=['date','opponent'])

In [78]:
merged

,actual_x,predicted_x,date,team_x,opponent_x,result_x,new_team_x,actual_y,predicted_y,team_y,opponent_y,result_y,new_team_y
0,L,W,2022-09-04,Arsenal,Manchester Utd,L,Arsenal,W,W,Manchester United,Arsenal,W,Manchester Utd
1,W,W,2022-09-18,Arsenal,Brentford,W,Arsenal,L,W,Brentford,Arsenal,L,Brentford
2,W,W,2022-10-01,Arsenal,Tottenham,W,Arsenal,L,W,Tottenham Hotspur,Arsenal,L,Tottenham
3,W,W,2022-10-09,Arsenal,Liverpool,W,Arsenal,L,W,Liverpool,Arsenal,L,Liverpool
4,W,W,2022-10-16,Arsenal,Leeds United,W,Arsenal,L,L,Leeds United,Arsenal,L,Leeds United
...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,L,L,2022-10-23,Wolverhampton Wanderers,Leicester City,L,Wolves,W,W,Leicester City,Wolves,W,Leicester City
156,D,D,2022-10-29,Wolverhampton Wanderers,Brentford,D,Wolves,D,W,Brentford,Wolves,D,Brentford
157,L,W,2022-11-05,Wolverhampton Wanderers,Brighton,L,Wolves,W,W,Brighton and Hove Albion,Wolves,W,Brighton and Hove Albion
158,L,L,2022-11-12,Wolverhampton Wanderers,Arsenal,L,Wolves,W,W,Arsenal,Wolves,W,Arsenal
